### Load dependencies


In [50]:
%pip install -q -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


### Load environment variables


In [25]:
import os
from dotenv import load_dotenv

load_dotenv()

True

### Set up vector store


In [ ]:
import json
from langchain_openai import AzureOpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document

embeddings = AzureOpenAIEmbeddings(
    api_key=os.environ["OPENAI_API_KEY"],
    azure_endpoint=os.environ["OPENAI_API_ENDPOINT"],
    deployment=os.environ["OPENAI_API_EMBEDDINGS_DEPLOYMENT"],
)

with open("./data/work_orders.json", "r") as f:
    work_orders = json.load(f)

documents = [Document(page_content=json.dumps(wo)) for wo in work_orders]

db = FAISS.from_documents(documents, embeddings)

### Import shared dependencies


In [ ]:
from langchain.tools import tool
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List, Optional

### Create query tools


In [ ]:
class WorkOrderSearch(BaseModel):
    issue: str = Field(
        ...,
        description="Issue reported by the maintenance tech to search work orders. For example, air entrapment or incomplete curing",
    )
    temporal: str = Field(
        ...,
        description="Temporal string that refers to the first or last `n` work orders. Options can be 'first', 'last' or 'all'",
    )
    n: Optional[int] = Field(..., description="Number of work orders requested")


@tool(args_schema=WorkOrderSearch)
def search_work_orders(issue: str, temporal: str, n: Optional[int] = 3) -> List:
    """Search for work orders where a similar issue was observed and/or resolved."""

    embedding = embeddings.embed_query(issue)
    results = db.similarity_search_with_score_by_vector(embedding, n)
    result_documents = [json.loads(result[0].page_content) for result in results]

    if temporal == "last":
        result_documents = sorted(
            result_documents, key=lambda doc: doc["date"], reverse=True
        )
    elif temporal == "first":
        result_documents = sorted(
            result_documents, key=lambda doc: doc["date"], reverse=False
        )

    return result_documents

In [ ]:
import pandas as pd
from pandasql import sqldf

parts = pd.read_json("./data/parts.json")


class PartsSearch(BaseModel):
    partName: Optional[str] = Field(
        ..., description="Name or description of the part and is a word or phrase"
    )
    partNumber: Optional[str] = Field(
        ..., description="Part number or ID. This is in the format `XX-00000`"
    )


@tool(args_schema=PartsSearch)
def search_parts(partName: Optional[str], partNumber: Optional[str]) -> List:
    """Search for a part, including its current inventory and location."""

    df = parts

    if partNumber is not None:
        return sqldf(f"SELECT * FROM df WHERE partNumber = '{partNumber}'", locals())
    elif partName is not None:
        return sqldf(f"SELECT * FROM df WHERE partName = '{partName}'", locals())
    else:
        return []

### Load RAG bot


In [ ]:
from rag_bot import RagBot

rag_bot = RagBot(tools=[search_parts, search_work_orders])

In [96]:
from langchain_openai import AzureChatOpenAI
from langchain.prompts import ChatPromptTemplate

class Grade(BaseModel):
    score: int = Field(..., description="Provide the score")
    explanation: str = Field(..., description="Explain your reasoning for the score")


def run_and_evaluate(query: str, expected: str, thread_id: str):
    response = rag_bot.invoke(
        query,
        thread_id,
    )
    
    # context = [
    #     message.content for message in response["messages"] if message.type == "tool"
    # ]
    answer = response["messages"][-1].content

    # source: https://smith.langchain.com/hub/langchain-ai/rag-answer-vs-reference
    grade_prompt_answer_accuracy = ChatPromptTemplate(
        [
            (
                "system",
                """You are a teacher grading a quiz. 

You will be given a QUESTION, the GROUND TRUTH (correct) ANSWER, and the STUDENT ANSWER. 

Here is the grade criteria to follow:

(1) Grade the student answers based ONLY on their factual accuracy relative to the ground truth answer. 

(2) Ensure that the student answer does not contain any conflicting statements.

(3) It is OK if the student answer contains more information than the ground truth answer, as long as it is factually accurate relative to the  ground truth answer.

Score:

A score of 1 means that the student's answer meets all of the criteria. This is the highest (best) score. 

A score of 0 means that the student's answer does not meet all of the criteria. This is the lowest possible score you can give.

Explain your reasoning in a step-by-step manner to ensure your reasoning and conclusion are correct. 

Avoid simply stating the correct answer at the outset.""",
            ),
            (
                "human",
                """QUESTION: {question}

GROUND TRUTH ANSWER: {correct_answer}

STUDENT ANSWER: {student_answer}""",
            ),
        ]
    )

    eval_llm = AzureChatOpenAI(
        api_key=os.environ["OPENAI_API_KEY"],
        api_version="2024-02-01",
        azure_deployment=os.environ["OPENAI_API_DEPLOYMENT"],
        azure_endpoint=os.environ["OPENAI_API_ENDPOINT"],
        streaming=False,
        temperature=0,
        verbose=False,
    ).with_structured_output(Grade)

    answer_grader = grade_prompt_answer_accuracy | eval_llm

    score = answer_grader.invoke(
        {
            "question": query,
            "correct_answer": expected,
            "student_answer": answer,
        }
    )

    return {
        "response": answer,
        "score": score,
    }

### Execute and test

In [97]:
import uuid
from pprint import pprint

thread_id = str(uuid.uuid4())

eval = run_and_evaluate(
    query="What are the last 2 work orders that show excessive mold wear?",
    expected="""Here are the last two work orders related to excessive mold wear:

1. **Work Order ID:** ISSUE-1017
   - **Date:** January 27, 2024
   - **Issue:** Excessive mold wear
   - **Solution:** Use high-quality mold materials and coatings. Regularly inspect and maintain the mold. Implement a preventive maintenance schedule to extend mold life.
   - **Replaced Parts:** HE-12345, CP-44556

2. **Work Order ID:** ISSUE-1013
   - **Date:** January 18, 2024
   - **Issue:** Mold surface damage
   - **Solution:** Regularly inspect and repair any damage to the mold surface. Use protective coatings to prevent further damage. Replace the mold if necessary to ensure product quality.
   - **Replaced Parts:** HE-12345
""",
    thread_id=thread_id,
)

pprint(eval)

eval = run_and_evaluate(
    query="What parts were replaced? What are their locations and inventory?",
    expected="""Here are the details for the replaced parts:

1. **Part Name:** Heating Element
   - **Part Number:** HE-12345
   - **Inventory:** 15 units
   - **Location:** Aisle 3, Shelf B

2. **Part Name:** Control Panel
   - **Part Number:** CP-44556
   - **Inventory:** 3 units
   - **Location:** Aisle 4, Shelf C
""",
    thread_id=thread_id,
)

pprint(eval)

{'response': 'Here are the last two work orders related to excessive mold '
             'wear:\n'
             '\n'
             '1. **Work Order ID:** ISSUE-1017\n'
             '   - **Date:** January 27, 2024\n'
             '   - **Issue:** Excessive mold wear\n'
             '   - **Solution:** Use high-quality mold materials and coatings. '
             'Regularly inspect and maintain the mold. Implement a preventive '
             'maintenance schedule to extend mold life.\n'
             '   - **Replaced Parts:** HE-12345, CP-44556\n'
             '\n'
             '2. **Work Order ID:** ISSUE-1013\n'
             '   - **Date:** January 18, 2024\n'
             '   - **Issue:** Mold surface damage\n'
             '   - **Solution:** Regularly inspect and repair any damage to '
             'the mold surface. Use protective coatings to prevent further '
             'damage. Replace the mold if necessary to ensure product '
             'quality.\n'
             '   - **Replac